In [7]:
import torch
print(f"get device name : {torch.cuda.get_device_name(0)}")
print(f"is available : {torch.cuda.is_available()}")
print(f"torch version : {torch.__version__}")

NameError: name '_C' is not defined

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F # 비선형 활성화 함수 출력에 사용 (relu, leaky_relu, softmax, log_softmax, tanh, sigmoid) batch_norm, layer_norm 도 여기서
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# LeNet-5 모델 정의
# 인스턴스: 클래스에 의해 생성된 객체 (실체화된 것)

class LeNet5(nn.Module):
    def __init__(self): # init 메소드: 클래스가 인스턴스화 될 때 호출되는 메소드
        super(LeNet5, self).__init__() # super는 부모 클래스를 참조하는 데 사용 (부모 클래스의 메소드 호출) -> 여기서는 부모 클래스인 nn.Module의 __init__을 호출
        # nn.Module의 __init__을 상속 받아서 호출하면 nn.Module에서 정의된 속성과 초기화 작업이 자식 클래스인 LeNet5에서도 수행됨
        # -> 이로써 코드의 중복을 피하고 유지보수를 용이하게 함
        # super(LeNet5, self)의 의미는 현재 클래스 이름이 LeNet5라는 것을 의미하고, self 객체가 속한 클래스가 LeNet5라는 것을 말함
        # self는 현재 인스턴스를 의미함. 만약 LeNet5 클래스로 객체화된 인스턴스가 있다면 그 인스턴스를 self라고 하는 것
        
        # Conv2d(in_channels = , out_channels =, kernel_size =, stride = , padding = )
        # kernel_size가 filter 크기
        # [batch_size, in_channels, H, W] -> [batch_size, out_channels, H - kernel_size + 1, W - kernel_size + 1]
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2) # 속성
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0) # 속성
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # FCs -> 선형적 신경망 레이어 1
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84) # FC 레이어 2
        self.fc3 = nn.Linear(84, 10) # FC 레이어 3

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 데이터 준비 (MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

# 모델, 손실 함수, 옵티마이저 정의
model = LeNet5()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# GPU 사용 가능 시 모델과 데이터를 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

# 모델 평가
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total} %')


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        
        # Encoder
        self.enc1 = self.contract_block(in_channels, 64, 7, 3)
        self.enc2 = self.contract_block(64, 128, 3, 1)
        self.enc3 = self.contract_block(128, 256, 3, 1)
        self.enc4 = self.contract_block(256, 512, 3, 1)
        
        # Decoder
        self.upconv4 = self.expand_block(512, 256, 3, 1)
        self.upconv3 = self.expand_block(256, 128, 3, 1)
        self.upconv2 = self.expand_block(128, 64, 3, 1)
        self.upconv1 = self.expand_block(64, out_channels, 7, 3)
    
    def __call__(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(F.max_pool2d(enc1, kernel_size=2, stride=2))
        enc3 = self.enc3(F.max_pool2d(enc2, kernel_size=2, stride=2))
        enc4 = self.enc4(F.max_pool2d(enc3, kernel_size=2, stride=2))
        
        # Decoder
        dec4 = self.upconv4(F.interpolate(enc4, scale_factor=2, mode='bilinear', align_corners=True))
        dec3 = self.upconv3(F.interpolate(dec4 + enc3, scale_factor=2, mode='bilinear', align_corners=True))
        dec2 = self.upconv2(F.interpolate(dec3 + enc2, scale_factor=2, mode='bilinear', align_corners=True))
        dec1 = self.upconv1(F.interpolate(dec2 + enc1, scale_factor=2, mode='bilinear', align_corners=True))
        
        return dec1

    def contract_block(self, in_channels, out_channels, kernel_size, padding):
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU()
        )
        return block
    
    def expand_block(self, in_channels, out_channels, kernel_size, padding):
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=padding),
            nn.ReLU()
        )
        return block


## Torch.nn 쓰지 않고 직접 생성한다면

- https://tutorials.pytorch.kr/beginner/nn_tutorial.html

In [1]:
import math
import torch

weights = torch.randn(784, 10) / math.sqrt(784) # Xavier initialization (randn로 784행 10열을 표준정규분포로부터 랜덤하게 생성 )
weights.requires_grad_() # weight에 대한 기울기(gradient 계산이 필요함을 알리는 설정, 이를 설정하면 자동으로 gradient를 계산해줌)
bias = torch.zeros(10, requires_grad =  True) # Bias로 0을 사용하되, gradient 계산이 필요함을 알리고자 requires_grad = True로 설정

In [13]:
# softmax 함수 직접 생성
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)
# torch.unsqueeze(): 1차원 텐서를 어느 위치에 차원 add할 것인지. 현재 weights 텐서는 (784, 10) 차원이므로 
# torch.squeeze(): 1차원 텐서를 어느 위치에서 remove할 것인지

# log_softmax 레이어만 통과하는 모델
def model(xb):
    return log_softmax(xb @ weights + bias) # @는 행렬곱을 말함 (그럼 행렬의 차원을 맞춰야겠지)

In [11]:
# squeeze, unsqueeze 이해
a = torch.randn((3, 2, 1))
print(a.shape)
print(a.squeeze(2).shape)
print(a.unsqueeze(-1).shape)

torch.Size([3, 2])
torch.Size([3, 2])
torch.Size([3, 2, 1])


In [12]:
a = torch.randn(3, 2)
print(a)
print(a.unsqueeze(-1))

tensor([[ 0.9695,  1.1148],
        [-1.4636,  0.9117],
        [-1.0689, -0.5011]])
tensor([[[ 0.9695],
         [ 1.1148]],

        [[-1.4636],
         [ 0.9117]],

        [[-1.0689],
         [-0.5011]]])


In [6]:
import torch

NameError: name '_C' is not defined

In [4]:
import torch
# import torch.nn
# import torch.optim as optim
# import torch.nn.functional as F

x = torch.tensor([1, 2, 3], requires_grad = True)

NameError: name '_C' is not defined

In [ ]:
import torch

: 